In [4]:
#!/usr/bin/env python
# coding: utf-8

#  Importar Paquetes

# Section[1]:


from CoachAIUtils import *
import re
import json
import IPython.display as ht
import pandas as pd
import wandb
from ipywidgets import *
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, LSTM, SimpleRNN, Dense, Flatten
from keras import optimizers
from keras import metrics
from keras.callbacks import TensorBoard
from time import time

jupy = True
debug = True

if jupy: get_ipython().magic('matplotlib inline')

#wandb.init(magic=True)
setpar = {
    "trainsetlist": "0,1",
    "trainyearlist": "2007,2008,2009,2010,2011,2012,2013,2014",
    "testsetlist": "0,1",
    "testyearlist": "2015,2016,2017,2018,2019",
    "modeltype": "LSTM",  # @param ["LSTM", "RNN", "Attention", "Red No Recurrente", "Boosting"]
    "eslabones": 3,  # @param {type:"slider", min:2, max:10, step:1}
    "look_back":  3,  # @param {type:"slider", min:2, max:100, step:1}
    "look_fwd": 0, # @param {type:"slider", min:0, max:10, step:1}
    "runs": 1, # @param {type:"slider", min:0, max:5, step:1}
    "overlap":  True,  # @param {type:"boolean"}
    "shuffle":  True,  # @param {type:"boolean"}
    "multiyear":  True,  # @param {type:"boolean"}
    "lota":  1.3,  # @param {type:"slider", min:0, max:10, step:0.1}
    "lotb":  1,  # @param {type:"slider", min:0, max:10, step:0.1}
    "lotc":  0.5,  # @param {type:"slider", min:0, max:10, step:0.1}
    "lotd":  0.1,  # @param {type:"slider", min:0, max:10, step:0.1}
    "grafancho":  12, # @param {type:"slider", min:3, max:100, step:1}
    "grafalto":  6, # @param {type:"slider", min:3, max:100, step:1}
    "epochs":  10,  # @param {type:"slider", min:0, max:500, step:10}
    "batch_size":  200,  # @param {type:"slider", min:0, max:10000, step:100}
    "validation_split":  0.2,  # @param {type:"slider", min:0, max:1, step:0.1}
    "optimizador":  "Adam",  # @param ["Adam", "SGD", "RMS", "Adagrad", "Adadelta", "Adamax", "Nadam"]
    "lr":  0.1,  # @param {type:"slider", min:0, max:1, step:0.001}
    "decay":  0.1,  # @param {type:"slider", min:0, max:1, step:0.001}
    "dropout":  0,  # @param {type:"slider", min:0, max:1, step:0.001}
    "recurrent_dropout":  0,  # @param {type:"slider", min:0, max:1, step:0.001}
    "determinista":  False}

savefich = ""
loadfich = ""
dire=".\\"
easetstrain = []
yearstrain = []
easetstest = []
yearstest = []

con = conectamysql("mysql+pymysql://{}:{}@{}/{}".format(user, passwd, host, db))
dfhead = pd.read_sql("select distinct EASetName from easet", con)
df = pd.read_sql("select distinct ﻿EASetName, Modifications from easettrades", con)
easetlist = df["﻿EASetName"].unique()


In [5]:
def cargaseldatos(ptrainsetlist, ptrainyearlist, ptestsetlist, ptestyearlist):
    global trd, loadedsets, easetstrain, yearstrain, easetstest, yearstest

    for y in ptrainsetlist.split(","):
        easetstrain.append(easetlist[int(y)])

    for y in ptrainyearlist.split(","):
        yearstrain.append(y)

    for y in ptestsetlist.split(","):
        easetstest.append(easetlist[int(y)])

    for y in ptestyearlist.split(","):
        yearstest.append(y)

    trd = np.array(np.zeros((len(easetstrain) + len(easetstest), 10000, 70)), dtype=object)
    dfmulti = ""
    for st in easetstrain:
        dfmulti += "'{}',".format(st)

    for st in easetstest:
        dfmulti += "'{}',".format(st)

    dfmulti = dfmulti[:-1]
    print("Cargando {}".format(dfmulti))
    dftodos = pd.read_sql("select * from easettrades where ﻿EASetName IN ({}) ORDER BY 1, 3 ".format(dfmulti), con)

    i = 0
    for st in dftodos["﻿EASetName"].unique():
        dfsingle = dftodos[dftodos["﻿EASetName"] == st]
        trd[i, 0:dfsingle.shape[0], 0:dfsingle.shape[1]] = dfsingle.values
        i += 1

    loadedsets = np.concatenate((easetstrain, easetstest))
    return "Cargado"


def muestrasets():
    print("Sets Disponibles:")

    i = 0
    s = "<table><tr>"
    for st in easetlist:
        s += "<td>{}</td><td>{}</td>".format(i, nombrebueno(df, st))
        if i % 4 == 0:
            s += "</tr><tr>"
        i += 1
    s += "</tr></table>"
    if jupy:
        ht.display(ht.HTML(s))
    else:
        print(s)

    faltansets = False
    for st in dfhead["EASetName"].unique():
        if len(df[df["﻿EASetName"] == st]) == 0:
            if faltansets is False:
                print("Sets Parseados pero no disponibles:")
                faltansets = True
            print(st)

    if faltansets is False:
        print("Ninguno")
    return

def create_dataset(dataset, pcol, plback=2, plfwd=0):
    datax, datay, testx, testy, testadd, desctrain, desctest = [], [], [], [], [], [], []
    cols = {"easet": 0, "ticket": 1, "apertura": 4, "cierre": 5, "lote": 8, "pl": 28, "acc": 29}
    salto = 1 if setpar["overlap"] else setpar["eslabones"]
    lookback = plback * salto
    lookahead = plfwd * salto

    tot = 1
    for i in range(len(dataset)):
        for j in range(0, len(dataset[i]) - lookback - lookahead, salto):
            a = dataset[i, range(j, j + lookback, salto), pcol]
            b = dataset[i, j + lookback + lookahead, pcol]
            if b == 0.0 :
                continue
            c = dataset[i, j + lookback + lookahead, (cols["easet"], cols["easet"], cols["ticket"], cols["apertura"], cols["cierre"],
                                                      cols["lote"], cols["pl"], cols["acc"])]
            c[6] += sum(dataset[i, j + lookback + lookahead - salto:j + lookback + lookahead, cols["pl"]])
            c[3] = fechabuena(c[3])
            ano = c[3][:4]
            c[4] = fechabuena(c[4])
            c[1] = nombrebueno(df, c[1])
            desc = c[1] + "/" + ano

            if (c[0] in easetstrain) and (ano in yearstrain):
                datax.append(a)
                datay.append(b)
                if desc not in desctrain:
                    desctrain.append(desc)
            elif (c[0] in easetstest) and (ano in yearstest):
                testx.append(a)
                testy.append(b)
                testadd.append(c)
                if desc not in desctest:
                    desctest.append(desc)
            tot += 1

    if debug:
        print("Sets con los que Entrenará:", desctrain)
        print("Sets con los que Graficará:", desctest)
    return np.array(datax), np.array(datay), np.array(testx), np.array(testy), np.array(testadd)

# In[5]:


def cargaparset(ptrainsetlist, ptrainyearlist, ptestsetlist, ptestyearlist,
        pmodeltype, peslabones, plook_back, plook_fwd, pruns, poverlap, pshuffle, pmultiyear, plota, plotb, plotc, plotd,
        pgrafancho, pgrafalto, pepochs, pbatch_size, pvalidation_split, poptimizador, plr, pdecay, pdropout,
        precurrent_dropout, ploadfich):
    global setpar, loadfich

    setpar = {"trainsetlist": ptrainsetlist, "trainyearlist": ptrainyearlist, "testsetlist": ptestsetlist, "testyearlist": ptestyearlist,"modeltype": pmodeltype, "eslabones": peslabones, "look_back": plook_back, "look_fwd": plook_fwd, "runs": pruns, "overlap": poverlap, "shuffle": pshuffle, "multiyear": pmultiyear,
        "lota": plota, "lotb": plotb, "lotc": plotc, "lotd": plotd, "grafancho": pgrafancho, "grafalto": pgrafalto, "epochs": pepochs, "batch_size": pbatch_size, "validation_split": pvalidation_split,
        "optimizador": poptimizador, "lr": plr, "decay": pdecay, "dropout": pdropout, "recurrent_dropout": precurrent_dropout}
    loadfich = ploadfich
    if ploadfich != "":
        portpar = json.load(open(dire + "setport-" + ploadfich + ".json"))
        print ("Cargado "+ ploadfich)

    return "Cargado"

def modela(px_train, py_train, pmodeltype="LSTM", pretseq=False):
    if debug:
        print(px_train.shape, py_train.shape)

    lmodel = Sequential()

    # Recurrent layer
    if pmodeltype == "LSTM":
        lmodel.add(
            LSTM(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=pretseq, dropout=setpar["dropout"],
                 recurrent_dropout=setpar["recurrent_dropout"]))
    elif pmodeltype == "RNN":
        lmodel.add(SimpleRNN(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=pretseq,
                             dropout=dropout, recurrent_dropout=recurrent_dropout))
    elif pmodeltype == "Attention":
        lmodel = keras.models.Sequential()
        lmodel.add(keras.layers.Embedding(input_dim=10000, output_dim=300, mask_zero=True))
        lmodel.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128, return_sequences=True)))
        lmodel.add(SeqSelfAttention(attention_activation='sigmoid'))
        lmodel.add(keras.layers.Dense(units=5))
        lmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'], )
        lmodel.summary()
    elif pmodeltype == "Red No Recurrente":
        lmodel.add(Dense(20, input_shape=(px_train.shape[1],), activation='sigmoid'))
        lmodel.add(Dense(5, activation='sigmoid'))
        # Error when checking target: expected activation_1 to have 3 dimensions, but got array with shape (3243, 5)
    elif pmodeltype == "Boosting":
        print("Boosting en desarrollo")
        exit(-1)
        # lmodel.add(LSTM(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=True, dropout=dropout,
        #                recurrent_dropout=recurrent_dropout))
        # lmodel.add(SeqSelfAttention(5, attention_activation='sigmoid'))
    elif pmodeltype == "Logistic Regression":
        clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(px_train, py_train)
        #clf.predict([:2, :])
        #clf.predict_proba(X[:2, :])
        #clf.score(X, y)

    lmodel.add(Activation('softmax'))
    # Compile the model
    if setpar["optimizador"] == "SGD":
        opt = optimizers.SGD(lr=lr, momentum=0.0, decay=decay, nesterov=False)
    elif setpar["optimizador"] == "Adam":
        opt = optimizers.Adam(lr=setpar["lr"], beta_1=0.9, beta_2=0.999, epsilon=None, decay=setpar["decay"], amsgrad=False)
    elif setpar["optimizador"] == "RMS":
        opt = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Adagrad":
        opt = optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Adadelta":
        opt = optimizers.Adadelta(lr=lr, rho=0.95, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Adamax":
        opt = optimizers.Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Nadam":
        optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=decay)

    lmodel.compile(optimizer=opt, loss='binary_crossentropy', metrics=[metrics.categorical_accuracy])

    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
    history = lmodel.fit(px_train, y_train, verbose=(0 + debug * 2), validation_split=setpar["validation_split"], epochs=setpar["epochs"],
                         batch_size=setpar["batch_size"], shuffle=setpar["shuffle"], callbacks=[tensorboard])
    print(lmodel.summary())
    return lmodel


# Datos de Set

## Selección de Datos
##  ___________________________
Sets a incluir en training:ptrainsetlist

Años a incluir en training:ptrainyearlist

Sets a incluir en test:ptestsetlist

Años a incluir en test:ptestyearlist
 
## Parámetros Set
## ______________
pmodeltype: Modelo

peslabones: Tipo de eslabon (eg: 5 mira los 5 ultimos trades)

plook_back: Longitud de la cadena (eg: 10 entrena con los ultimos 10 eslabones)

plook_fwd: Lag de la predicción (0 intenta predecir siguiente eslabon, 1 dos eslabones despues...)

pruns: Numero de veces que entrenamos

Eslabones solapados o secuenciales:poverlap

Si marcas multiyear, habra un grafico por set y no por año: pmultiyear

Nuevo lotaje:plota, plotb, plotc, plotd

Tamaño Gráficos:pgrafancho, pgrafalto
 
## Parametros de los modelos:
## __________________________
Número de pasadas:pepochs

Número de elementos en cada pasada:pbatch_size

% de los datos que se usan para validación:pvalidation_split

Optimizador: poptimizador

Learning Rate: Tamaño de la actualizacion en gradient descent:plr

Reduccion progresiva del learning rate:pdecay

Celdas a desactivar cada pasada del estado oculto para prevenir overfitting:pdropout

Celdas a desactivar de la "memoria" para prevenir overfitting:precurrent_dropout
 

In [6]:
interact_manual(cargaparset,
                ptrainsetlist= setpar["trainsetlist"],
                ptrainyearlist= setpar["trainyearlist"],
                ptestsetlist= setpar["testsetlist"],
                ptestyearlist= setpar["testyearlist"],
                pmodeltype = setpar["modeltype"],  # @param ["LSTM", "RNN", "Attention", "Red No Recurrente", "Boosting"]
                peslabones = setpar["eslabones"],  # @param {type:"slider", min:2, max:10, step:1}
                plook_back = setpar["look_back"],  # @param {type:"slider", min:2, max:100, step:1}
                plook_fwd = setpar["look_fwd"],  # @param {type:"slider", min:0, max:10, step:1}
                pruns = setpar["runs"],  # @param {type:"slider", min:0, max:10, step:1}
                poverlap = setpar["overlap"],  # @param {type:"boolean"}
                pshuffle = setpar["shuffle"],  # @param {type:"boolean"}
                pmultiyear = setpar["multiyear"],  # @param {type:"boolean"}
                plota = setpar["lota"],  # @param {type:"slider", min:0, max:10, step:0.1}
                plotb = setpar["lotb"],  # @param {type:"slider", min:0, max:10, step:0.1}
                plotc = setpar["lotc"],  # @param {type:"slider", min:0, max:10, step:0.1}
                plotd = setpar["lotd"],  # @param {type:"slider", min:0, max:10, step:0.1}
                pgrafancho = setpar["grafancho"],  # @param {type:"slider", min:3, max:100, step:1}
                pgrafalto = setpar["grafalto"],  # @param {type:"slider", min:3, max:100, step:1}
                pepochs = setpar["epochs"],  # @param {type:"slider", min:0, max:500, step:10}
                pbatch_size = setpar["batch_size"],  # @param {type:"slider", min:0, max:10000, step:100}
                pvalidation_split = setpar["validation_split"],  # @param {type:"slider", min:0, max:1, step:0.1}
                poptimizador = setpar["optimizador"],  # @param ["Adam", "SGD", "RMS", "Adagrad", "Adadelta", "Adamax", "Nadam"]
                plr = setpar["lr"],  # @param {type:"slider", min:0, max:1, step:0.001}
                pdecay = setpar["decay"],  # @param {type:"slider", min:0, max:1, step:0.001}
                pdropout = setpar["dropout"],  # @param {type:"slider", min:0, max:1, step:0.001}
                precurrent_dropout = setpar["recurrent_dropout"],  # @param {type:"slider", min:0, max:1, step:0.001}
                ploadfich = loadfich)

print("Configuraciones de Sets")
for parfich in os.listdir(dire):
    if re.match("setpar.*.json", parfich):
        print(parfich[7:-5])


interactive(children=(Text(value='0,1', description='ptrainsetlist'), Text(value='2007,2008,2009,2010,2011,201…

Configuraciones de Sets
17oct19
18oct19
20Oct19


## Si no sale nada, dar a Run All Above
## Despues de cargar una configuracion, dar al play en la celda de parametros para refrescar

In [7]:
if setpar["determinista"]:
    hacerdeterminista()

cargaseldatos(setpar["trainsetlist"], setpar["trainyearlist"], setpar["testsetlist"], setpar["testyearlist"])

trd = eslabones2numbers(trd)

if setpar["look_back"]:
    X_train, y_train, X_test, y_test, y_add = create_dataset(trd, 17 + setpar["eslabones"], plback=setpar["look_back"], plfwd=setpar["look_fwd"])
    if len(y_train) == 0:
        print("No hay datos de entrenamiento, revisa sets y años pls")
        exit(0)

    if len(y_test) == 0:
        print("No hay datos de test, revisa sets y años pls")
        exit(0)

#np.savetxt("c:\\users\\diego\\documents\\coachai\\Eslabones{}x{}.csv".format(eslabones, look_back),X_train.astype("int"), delimiter=";", fmt="%d")

    X_train = to_categorical(X_train) if setpar["modeltype"] == "LSTM" else X_train
    y_train = to_categorical(y_train)
    retseq = False
else:
    X_train = to_categorical(trd[:, :9999, 17 + eslabones])
    y_train = to_categorical(trd[:, 1:, 17 + eslabones])
    retseq = True

model = modela(X_train, y_train, pmodeltype=setpar["modeltype"])

def grabafich(psavefich):
    global savefich, dire

    savefich = psavefich
    if psavefich != "":
        savejson(setpar, dire + "setpar-" + psavefich)
        model.save(dire + psavefich + ".h5")
        print("Grabado " + psavefich)

    return "Grabado"



Cargando 'EA-A1v521012174692en3','EA-A1v521013202856uj3','EA-A1v521012174692en3','EA-A1v521013202856uj3'
Sets con los que Entrenará: ['EA-A1-v5210-1217-4692-en-3/2007', 'EA-A1-v5210-1217-4692-en-3/2008', 'EA-A1-v5210-1217-4692-en-3/2009', 'EA-A1-v5210-1217-4692-en-3/2010', 'EA-A1-v5210-1217-4692-en-3/2011', 'EA-A1-v5210-1217-4692-en-3/2012', 'EA-A1-v5210-1217-4692-en-3/2013', 'EA-A1-v5210-1217-4692-en-3/2014', 'EA-A1-v5210-1320-2856-uj-3/2007', 'EA-A1-v5210-1320-2856-uj-3/2008', 'EA-A1-v5210-1320-2856-uj-3/2009', 'EA-A1-v5210-1320-2856-uj-3/2010', 'EA-A1-v5210-1320-2856-uj-3/2011', 'EA-A1-v5210-1320-2856-uj-3/2012', 'EA-A1-v5210-1320-2856-uj-3/2013', 'EA-A1-v5210-1320-2856-uj-3/2014']
Sets con los que Graficará: ['EA-A1-v5210-1217-4692-en-3/2015', 'EA-A1-v5210-1217-4692-en-3/2016', 'EA-A1-v5210-1320-2856-uj-3/2015', 'EA-A1-v5210-1320-2856-uj-3/2016']
(1286, 3, 5) (1286, 5)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast ins

In [8]:
interact_manual(grabafich,
                psavefich= savefich)


interactive(children=(Text(value='', description='psavefich'), Button(description='Run Interact', style=Button…

<function __main__.grabafich>